In [ ]:
#install dependencies
%pip install requests pandas

In [ ]:
import requests
import pandas as pd

# Function to fetch driver data
def fetch_driver_data():
    url = 'https://api.openf1.org/v1/drivers'
    response = requests.get(url)
    if response.status_code == 200:
        try:
            return pd.DataFrame(response.json())
        except ValueError as e:
            print("Error decoding JSON:", e)
            return pd.DataFrame()
    else:
        print(f"Failed to fetch driver data: {response.status_code}")
        return pd.DataFrame()

# Function to fetch session data
def fetch_session_data():
    url = 'https://api.openf1.org/v1/sessions'
    response = requests.get(url)
    if response.status_code == 200:
        try:
            return pd.DataFrame(response.json())
        except ValueError as e:
            print("Error decoding JSON:", e)
            return pd.DataFrame()
    else:
        print(f"Failed to fetch session data: {response.status_code}")
        return pd.DataFrame()

# Updated function to fetch lap data including specific attributes
def fetch_lap_data(driver_number, session_key):
    url = f'https://api.openf1.org/v1/laps?driver_number={driver_number}&session_key={session_key}'
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            lap_df = pd.DataFrame(data)
            # Select relevant attributes for lap prediction
            relevant_lap_attributes = [
                'lap_number', 'lap_duration', 'duration_sector_1', 
                'duration_sector_2', 'duration_sector_3', 
                'i1_speed', 'i2_speed', 'st_speed', 'is_pit_out_lap'
            ]
            lap_df = lap_df[relevant_lap_attributes]
            return lap_df
        except ValueError as e:
            print("Error decoding JSON:", e)
            return pd.DataFrame()
    else:
        print(f"Failed to fetch lap data: {response.status_code}")
        return pd.DataFrame()

# Updated function to fetch stint data including specific attributes
def fetch_stint_data(driver_number, session_key):
    url = f'https://api.openf1.org/v1/stints?driver_number={driver_number}&session_key={session_key}'
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            stint_df = pd.DataFrame(data)
            # Select relevant attributes for lap prediction
            relevant_stint_attributes = [
                'stint_number', 'compound', 'tyre_age_at_start', 
                'lap_start', 'lap_end'
            ]
            stint_df = stint_df[relevant_stint_attributes]
            return stint_df
        except ValueError as e:
            print("Error decoding JSON:", e)
            return pd.DataFrame()
    else:
        print(f"Failed to fetch stint data: {response.status_code}")
        return pd.DataFrame()

# Fetch driver and session data
drivers = fetch_driver_data()
print("Driver Data:")
print(drivers.head())

sessions = fetch_session_data()
print("Session Data:")
print(sessions.head())

# Integrate new data fetching functions
all_laps = pd.DataFrame()
all_stints = pd.DataFrame()

# Loop through drivers and sessions to fetch lap and stint data
for index, row in drivers.iterrows():
    laps = fetch_lap_data(row['driver_number'], row['session_key'])
    stints = fetch_stint_data(row['driver_number'], row['session_key'])
    
    # Verify lap and stint data before concatenating
    if not laps.empty:
        print(f"Lap Data for Driver {row['driver_number']} in Session {row['session_key']}:")
        print(laps.head())
    else:
        print(f"No lap data found for Driver {row['driver_number']} in Session {row['session_key']}.")
    
    if not stints.empty:
        print(f"Stint Data for Driver {row['driver_number']} in Session {row['session_key']}:")
        print(stints.head())
    else:
        print(f"No stint data found for Driver {row['driver_number']} in Session {row['session_key']}.")

    # Combine lap and stint data with the main dataset
    all_laps = pd.concat([all_laps, laps], ignore_index=True)
    all_stints = pd.concat([all_stints, stints], ignore_index=True)

print("All Lap Data:")
print(all_laps.head())  # Verify lap data for relevant attributes

print("All Stint Data:")
print(all_stints.head())  # Verify stint data for tire compounds and stint details

# Merge all fetched data to form a comprehensive dataset
# This merge should be done carefully considering lap ranges and other identifiers
final_data = pd.merge(all_laps, all_stints, how='left', left_on=['lap_number'], right_on=['lap_start'])
print("Final Merged Data:")
print(final_data.head())  # Inspect the merged data for completeness